Salency maps para NLP utilizando AllenNLP
=========================================

Introducción
------------

AllenNLP es un framework general de aprendizaje profundo para NLP, establecido por el mundialmente famoso Allen Institute for AI Lab. Contiene modelos de referencia de última generación que se ejecutan sobre el `PyTorch`. AllenNLP es una librería que ademas busca implementar abstracciones que permitan el rápido desarrollo de modelos y reutilización de componentes al despegarse de detalles de implementación de cada modelo.

En este ejemplo, veremos como utilizar esta librería para generar salency maps utilizando los gradientes de las prediciones. Esto nos permita interpretar las predicciones de nuestros modelos basados en `transformers`.

### Para ejecutar este notebook

Para ejecutar este notebook, instale las siguientes librerias:

In [ ]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Datasets/mascorpus/tweets_marketing.csv \
    --quiet --no-clobber --directory-prefix ./Datasets/mascorpus/

!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/TextDataset.py \
    --quiet --no-clobber --directory-prefix ./Utils/
    
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/docs/nlp/neural/allennlp_interpret.txt \
    --quiet --no-clobber
%pip install -r allennlp_interpret.txt --quiet

Si ejecuta en Google Colab, adicionalmente deberá cambiar la version de la libraria `google-cloud-storage`:

In [ ]:
%pip install -U google-cloud-storage==1.40.0 --quiet

Descargaremos un modelo previamente entrenando el el problema de clasificación de Tweets:

In [3]:
!wget https://santiagxf.blob.core.windows.net/public/models/tweet_classification_bert.zip --no-clobber --quiet
!unzip -qq tweet_classification_bert.zip

Cargamos el set de datos

In [1]:
import pandas as pd

tweets = pd.read_csv('Datasets/mascorpus/tweets_marketing.csv')

Cargando un modelo entreando con Transformers en AllenNLP
---------------------------------------------------------

`allennlp` es un framework compatible con la libraría `transformers` lo cual resulta atractivo a la hora de utilizar modelos que son entrenados en una para luego llevarlo a la otra. Veamos entonces como podemos hacer para cargar el modelo que tenemos previamente entrenado para la clasificación de tweets utilizando una arquitectura `BERT` dentro de este framework. En particular, nuestro modelo se persistió en el directorio "tweet_classification".

In [2]:
model_name = "tweet_classification_bert"

### Creando un objeto Model

Importamos algunos elementos que necesitaremos

In [3]:
from typing import Dict, Iterable, List

from allennlp.common import Params
from allennlp.data import DatasetReader, Instance, Batch
from allennlp.data.fields import Field, LabelField, TextField
from allennlp.data.token_indexers import TokenIndexer
from allennlp.data.tokenizers import Tokenizer
from allennlp.data.vocabulary import PreTrainedTokenizer, Vocabulary
from allennlp.models import BasicClassifier, Model
from allennlp.modules.token_embedders import PretrainedTransformerEmbedder
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.data.tokenizers.pretrained_transformer_tokenizer import PretrainedTransformerTokenizer
from allennlp.data.token_indexers.pretrained_transformer_indexer import PretrainedTransformerIndexer
from allennlp.modules.seq2vec_encoders.bert_pooler import BertPooler

Cargaremos todos los elementos que son necesarios para utilizar esta libreria. Todos ellos son generados a partir del modelo que persistimos en `transformers`. La utilidad de cada uno de estos módulos esta fuera del alcance de este curso pero recomendamos revisar la documentación de AllenNLP para más información sobre cual es su rol.

In [4]:
transformer_vocab = Vocabulary.from_pretrained_transformer(model_name)
transformer_tokenizer = PretrainedTransformerTokenizer(model_name)
transformer_encoder = BertPooler(model_name)

token_indexer = PretrainedTransformerIndexer(model_name)

Some weights of the model checkpoint at tweet_classification_bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


> ¿Notan el mensaje de advertencia? Lo correjiremos mas adelante.

In [5]:
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders.pretrained_transformer_embedder import PretrainedTransformerEmbedder

token_embedder = BasicTextFieldEmbedder({ "tokens": PretrainedTransformerEmbedder(model_name) })

Creamos el modelo a partir de todos los componentes que cargamos anteriormente:

In [6]:
model = BasicClassifier(vocab=transformer_vocab, text_field_embedder=token_embedder, seq2vec_encoder=transformer_encoder, dropout=0.1, num_labels=7)

A continuación, cargaremos los pesos del clasificador.

In [7]:
from transformers import BertForSequenceClassification
classifier = BertForSequenceClassification.from_pretrained(model_name)

In [8]:
model._classification_layer.weight = classifier.classifier.weight
model._classification_layer.bias = classifier.classifier.bias

Configuremos el modelo para trabajar en modo `inferencia`:

In [9]:
_ = model.eval()

### Creamos un DatasetReader

AllenNLP utiliza un objeto llamado `DatasetReader` que le permite crear `Instance`'s de datos que son suministradas al modelo. Esta abstracción permite realizar cualquier preprocesamiento que es necesario antes de enviar los datos al modelo. Debemos generar nuestro propia implementación para el caso de clasificación utilizando un modelo basado en *transformers*. La siguiente clase realiza esto: 

In [10]:
from allennlp.data import DatasetReader

In [11]:
class ClassificationTransformerReader(DatasetReader):
    def __init__(
        self,
        tokenizer: Tokenizer,
        token_indexer: TokenIndexer,
        max_tokens: int,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.tokenizer = tokenizer
        self.token_indexers: Dict[str, TokenIndexer] = { "tokens": token_indexer }
        self.max_tokens = max_tokens

    def text_to_instance(self, text: str, label: str = None) -> Instance:
        tokens = self.tokenizer.tokenize(text)
        if self.max_tokens:
            tokens = tokens[: self.max_tokens]
        
        fields: Dict[str, Field] = { }
        fields["tokens"] = TextField(tokens, self.token_indexers)
            
        if label:
            fields["label"] = LabelField(label)
            
        return Instance(fields)

Instanciamos el `DatasetReader`

In [12]:
dataset_reader = ClassificationTransformerReader(tokenizer=transformer_tokenizer, 
                                                 token_indexer=token_indexer, 
                                                 max_tokens=400)

Interpretando nuestras predicciones
-----------------------------------

Una vez que tenemos nuestro modelo correctamente cargado, veamos como podemos interpretar una predicción computando el salency map a partir de los gradientes.

In [17]:
from allennlp.interpret.saliency_interpreters import SimpleGradient, IntegratedGradient, SmoothGradient
from allennlp.predictors import Predictor, TextClassifierPredictor

predictor = TextClassifierPredictor(model, dataset_reader)
interpreter = SmoothGradient(predictor)

Busquemos un tweet para interpretar:

In [21]:
sample_text_idx = 1522
sample_text = tweets['TEXTO'][sample_text_idx]
sample_label = tweets['SECTOR'][sample_text_idx]

print("Texto:", sample_text, "\Sector:", sample_label)

Texto: @HyundaiPeru con Grupo Primax realiza este verano servicios de Inspección Digital Gratuita a vehículos Hyundai en e… https://t.co/TZ4XFziOd3 \Sector: AUTOMOCION


Calculemos los gradientes para cada token:

In [22]:
import numpy as np

In [57]:
interpretation = interpreter.saliency_interpret_from_json({"sentence": sample_text })
outputs = predictor.predict(sample_text)
grads = np.array(interpretation['instance_1']['grad_input_1'])
probs = np.array(outputs['probs'])

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


In [24]:
outputs.keys()

dict_keys(['logits', 'probs', 'token_ids', 'label', 'tokens'])

Recordemos que en el conjunto de datos de entrenamiento, las etiquetas se distribuyen como sigue:

```
{
    'ALIMENTACION',
    'AUTOMOCION',
    'BANCA',
    'BEBIDAS',
    'DEPORTES',
    'RETAIL',
    'TELCO'
}
```

Podemos graficar los resultados utilizando un mapa de calor marcando con colores más intensos aquellos tokens que tienen mayor impacto en las predicciones:

In [25]:
import numpy as np
from eli5.base import Explanation, TargetExplanation, FeatureWeights, FeatureWeight, WeightedSpans, DocWeightedSpans

In [47]:
from typing import Tuple, List

def locate_token(token: str, token_idx: int, tokenized_text: List[str]) -> List[Tuple[int, int]]:
  str_to_token = ' '.join(tokenized_text[:token_idx + 1]).replace(" ##", '')
  str_to_token = str_to_token.replace("[CLS] ", '')
  striped_token = token.replace("##", '')

  return [tuple([len(str_to_token) - len(striped_token) - 1, len(str_to_token) - 1])]


In [58]:
from numpy.ma.core import array
expl = Explanation(estimator="transformer",
                   description="NLP transformer explanation",
                   targets=[
                      TargetExplanation(target="AUTOMOCION",
                                        feature_weights=FeatureWeights(pos=[FeatureWeight("Highlighted in text", weight=np.array(grads).sum(), value=0)], neg=[]),
                                        proba=np.array(outputs['probs']).max(),
                                        weighted_spans=WeightedSpans(
                                            [DocWeightedSpans(sample_text, 
                                                              spans=[(token, locate_token(token, idx, outputs['tokens']), grads[idx]) for idx, token in enumerate(outputs['tokens'])])
                                            ]
                                        ))
                   ])

In [49]:
from IPython.display import HTML
from eli5.formatters import format_as_html, fields

In [59]:
HTML(format_as_html(expl))

In [60]:
from numpy.ma.core import array
expl = Explanation(estimator="transformer",
                   description="NLP transformer explanation",
                   is_regression=False,
                   targets=[
                      TargetExplanation(target="AUTOMOCION",
                                        feature_weights=FeatureWeights(pos=[FeatureWeight(token, grads[idx]) for idx, token in enumerate(outputs['tokens'])],
                                                                       neg=[]),
                                        proba=np.array(outputs['probs']).max(),
                                        score=np.array(outputs['probs']).argmax(),
                                        weighted_spans=WeightedSpans(
                                            [DocWeightedSpans(sample_text, 
                                                              spans=[(token, locate_token(token, idx, outputs['tokens']), grads[idx]) for idx, token in enumerate(outputs['tokens'])])
                                            ]
                                        ))
                   ])

In [61]:
HTML(format_as_html(expl, force_weights=True))

Weight?,Feature
+0.011,[CLS]
+0.000,@
+0.000,hyun
+0.029,##da
+0.062,##ipe
+0.029,##ru
+0.000,con
+0.044,grupo
+0.000,prima
+0.000,##x
